In [1]:
import sys
from collections import defaultdict
from itertools import combinations

import numpy as np

from src.cfd import CFD
from src.dataset import X_TIME, load_expanded_dataset
from src.utils import deconvolve_precision, PLANES

PWD = '../..'
sys.path.append(PWD)

IMAGES_DIR = PWD + '/data/images/cfd_deconvolution/'

In [2]:
CFD_THRESHOLD = 0.2
CFD_N_BASELINE = 6

cfd = CFD(n_baseline=CFD_N_BASELINE, threshold=CFD_THRESHOLD)

_, dataset_wav, dataset_t0, dataset_t_cfd = load_expanded_dataset(PWD)

# Add t0 to CFD timestamps

In [3]:
cfd_timestamps = {key: dataset_t0[key] + dataset_t_cfd[key] for key in dataset_t0.keys()}

# Find mutual precisions

In [4]:
mutual_precisions = {}

for p_ch1, p_ch2 in combinations(cfd_timestamps.keys(), 2):
    if p_ch1[1] == p_ch2[1]:  # Only for corresponding channels
        ch1_timestamps, ch2_timestamps = cfd_timestamps[p_ch1], cfd_timestamps[p_ch2]
        differences = [abs(ch1_t - ch2_t) for ch1_t, ch2_t in zip(ch1_timestamps, ch2_timestamps) if
                       not np.isnan(ch1_t) and not np.isnan(ch2_t)]

        mutual_precisions[(p_ch1, p_ch2)] = np.std(differences)

In [5]:
channel_mutual_precisions = defaultdict(dict)
for ((x_p, x_ch), (y_p, y_ch)), precision in mutual_precisions.items():
    assert x_ch == y_ch
    channel_mutual_precisions[x_ch][(x_p, y_p)] = precision

In [6]:
for ch, data in channel_mutual_precisions.items():
    for (p_1, p_2), prec in data.items():
        print(f'ch {ch:2}: (p{p_1} vs p{p_2}): {prec:0.3f}')

ch  2: (p1 vs p2): 0.056
ch  2: (p1 vs p3): 0.070
ch  2: (p2 vs p3): 0.072
ch 11: (p1 vs p2): 0.063
ch 11: (p1 vs p3): 0.077
ch 11: (p2 vs p3): 0.081


# Deconvolve

In [7]:
deconvolved_precisions = {}
for ch, prec_dict in channel_mutual_precisions.items():
    for p in PLANES:
        deconvolved_precisions[(p, ch)] = deconvolve_precision(p, prec_dict)

In [8]:
for (p, ch), prec in deconvolved_precisions.items():
    print(f'(p {p}, ch {ch:2}): {prec:0.3f}')

(p 1, ch  2): 0.053
(p 2, ch  2): 0.059
(p 3, ch  2): 0.083
(p 1, ch 11): 0.057
(p 2, ch 11): 0.069
(p 3, ch 11): 0.092
